#Set-Up

In [3]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Lego Challenge Problem Code

In [5]:
from pyomo.environ import *

# Initialize the model
model = ConcreteModel()

# Sets
pieces = ["Flat 2x2", "Thick 1x4", "Thick 2x4", "Flat 1x8"]  # Piece types
objects = ["Sled", "Casket", "Bookshelf", "Cat Climber", "Coffee Table", "Moose Head", "Balance Beam", "Step Stool"]  # Objects to build

# Parameters
inventory_available = {"Flat 2x2": 15, "Thick 1x4": 8, "Thick 2x4": 6, "Flat 1x8": 7}  # Inventory constraints
requirements = {  # Pieces required to build each object
    "Sled": [2, 0, 1, 2],
    "Casket": [4, 2, 1, 2],
    "Bookshelf": [6, 2, 0, 0],
    "Cat Climber": [2, 0, 1, 1],
    "Coffee Table": [4, 0, 1, 0],
    "Moose Head": [2, 0, 1, 1],
    "Balance Beam": [0, 2, 0, 1],
    "Step Stool": [2, 0, 1, 0],
}
profits = {  # Profit for each object
    "Sled": 150,
    "Casket": 200,
    "Bookshelf": 130,
    "Cat Climber": 100,
    "Coffee Table": 90,
    "Moose Head": 80,
    "Balance Beam": 60,
    "Step Stool": 40,
}

# Variables
model.x = Var(objects, domain=NonNegativeIntegers)  # Number of each object to build

# Objective: Maximize profit
model.profit = Objective(expr=sum(model.x[obj] * profits[obj] for obj in objects), sense=maximize)

# Constraints
# 1. Do not exceed inventory limits for each piece type
def inventory_constraint_rule(model, p):
    piece_index = pieces.index(p)  # Get the index of the piece type
    return sum(model.x[obj] * requirements[obj][piece_index] for obj in objects) <= inventory_available[p]

model.inventory_constraints = Constraint(pieces, rule=inventory_constraint_rule)

# Solve the model
solver = SolverFactory('cbc')  # You can use other solvers like 'glpk' if available
results = solver.solve(model)

# Display results
if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
    print("Optimal Production Plan:")
    for obj in objects:
        print(f"{obj}: {model.x[obj]() if model.x[obj]() else 0}")
    total_profit = sum(model.x[obj]() * profits[obj] for obj in objects)
    print(f"\nTotal Profit: ${total_profit}")
else:
    print("Optimization problem is unbounded or infeasible. Check constraints.")


Optimal Production Plan:
Sled: 0
Casket: 0
Bookshelf: 0
Cat Climber: 3.0
Coffee Table: 2.0
Moose Head: 0
Balance Beam: 4.0
Step Stool: 0

Total Profit: $720.0
